<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" style="height:450px;" width=500/></p>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>
<h3 style="text-align: center;"><b>Продвинутый поток (часть 2). Весна 2021</b></h3>

<h1 style="text-align: center;"><b>Language modeling.</b></h1>

Для начала загрузим датасет, состоящий из сэмплов кода на языке Python. Датасет представлен гитхабом. [Про датасет](https://github.blog/2019-09-26-introducing-the-codesearchnet-challenge/).

Для препроцессинга будем использовать уже известную нам библиотеку `datasets` от Huggingface.

In [ ]:
#!pip install -q datasets

     |████████████████████████████████| 194kB 21.9MB/s 
     |████████████████████████████████| 112kB 51.4MB/s 
     |████████████████████████████████| 245kB 58.4MB/s 


In [ ]:
#!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
#!unzip -p python.zip python/final/jsonl/train/python_train_0.jsonl.gz > train.jsonl.gz
#!unzip -p python.zip python/final/jsonl/test/python_test_0.jsonl.gz > test.jsonl.gz

--2021-03-06 10:28:54--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.70.198
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.70.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  95.2MB/s    in 9.4s    

2021-03-06 10:29:04 (95.0 MB/s) - ‘python.zip’ saved [940909997/940909997]



In [ ]:
# decompress this gzip file
#!gzip -d train.jsonl.gz
#!gzip -d test.jsonl.gz

Загружать датасеты можно не только из хаба, но и из диска. Для этого достаточно указать формат и путь до файла.

In [1]:
from datasets import load_dataset  
dataset = load_dataset(
    "json",
    data_files=[
        "train.jsonl",
    ],
)

Using custom data configuration default-d7a15e7d6464b3a9
Reusing dataset json (C:\Users\BIT\.cache\huggingface\datasets\json\default-d7a15e7d6464b3a9\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 30000
    })
})

Ограничим число уникальных слов до `40000`.

In [4]:
from collections import Counter


vocab_size = 40000
stats = Counter()

for item in dataset["train"]: # итерируемся по датасету
    stats.update(item["code_tokens"]) # добавляем 1 к кол-ву соответвующего символа
tokens = dict(stats.most_common(vocab_size)).keys() # выбираем 40000 самых популярных слов

Посмотрим на самые часто встречающиеся слова

In [5]:
stats.most_common(20)

[('(', 299562),
 (')', 299456),
 ('.', 283659),
 (',', 267888),
 ('=', 238494),
 (':', 176959),
 ('[', 99465),
 (']', 99448),
 ('self', 71449),
 ('if', 65763),
 ('return', 40852),
 ('None', 37311),
 ('def', 32278),
 ('in', 29951),
 ('*', 21773),
 ('not', 21291),
 ('name', 21205),
 ('ret', 20862),
 ('1', 19494),
 ('for', 19364)]

Добавим служебные токены

In [6]:
PAD = 0
UNK = 1
EOS = 2

token2idx = {"[PAD]": 0, "[UNK]": 1, "[EOS]": 2}

for idx, token in enumerate(tokens):
    token2idx[token] = idx + 3 # token2idx словарь (слово вокабуляра:его индекс)

In [11]:
token2idx

{'[PAD]': 0,
 '[UNK]': 1,
 '[EOS]': 2,
 '(': 3,
 ')': 4,
 '.': 5,
 ',': 6,
 '=': 7,
 ':': 8,
 '[': 9,
 ']': 10,
 'self': 11,
 'if': 12,
 'return': 13,
 'None': 14,
 'def': 15,
 'in': 16,
 '*': 17,
 'not': 18,
 'name': 19,
 'ret': 20,
 '1': 21,
 'for': 22,
 '0': 23,
 'else': 24,
 '{': 25,
 '}': 26,
 'False': 27,
 '==': 28,
 'is': 29,
 'format': 30,
 'True': 31,
 'kwargs': 32,
 'path': 33,
 '+': 34,
 'tf': 35,
 'raise': 36,
 '-': 37,
 'data': 38,
 'key': 39,
 'get': 40,
 'x': 41,
 'and': 42,
 'append': 43,
 'len': 44,
 'os': 45,
 'isinstance': 46,
 'i': 47,
 'except': 48,
 'elif': 49,
 'try': 50,
 'value': 51,
 'hparams': 52,
 'result': 53,
 'np': 54,
 'salt': 55,
 '2': 56,
 'args': 57,
 'or': 58,
 'log': 59,
 'utils': 60,
 'as': 61,
 'str': 62,
 "'comment'": 63,
 'join': 64,
 '__salt__': 65,
 '%': 66,
 '!=': 67,
 'dtype': 68,
 '+=': 69,
 "''": 70,
 'cmd': 71,
 'index': 72,
 "'result'": 73,
 'line': 74,
 'int': 75,
 'k': 76,
 'ValueError': 77,
 'shape': 78,
 'list': 79,
 'profile': 80,
 

Переведем токены в индексы

In [7]:
def encode(token): # возвращает индекс токена из словаря
    if token in token2idx.keys():
        return token2idx[token]
    return UNK # UNK = 1

In [15]:
dataset = dataset.map( # превращаем токены в индексы и записываем в словарь features датасета
    lambda item: {
        "features": [encode(token) for token in item["code_tokens"]] + [EOS]
    }
)

  0%|          | 0/30000 [00:00<?, ?ex/s]

## N-gram

 Начнём с простейшей модели. Она основывается на статистическом методе. Итак, в языковом моделировании мы хотим максимизировать вероятность нашего текста по мнению модели, то есть:
 $$
\mathrm{P}(\mathrm{W})=\mathrm{P}\left(\mathrm{w}_{1}, \mathrm{w}_{2}, \mathrm{w}_{3}, \mathrm{w}_{4}, \mathrm{w}_{5} \ldots \mathrm{w}_{\mathrm{n}}\right)
$$


Вспомним, что можно переписать:

$$
P\left(x_{1}, x_{2}, x_{3}, \ldots, x_{n}\right)=P\left(x_{1}\right) P\left(x_{2} \mid x_{1}\right) P\left(x_{3} \mid x_{1}, x_{2}\right) \ldots P\left(x_{n} \mid x_{1}, \ldots, x_{n-1}\right)
$$

P(Я хочу спать) = P(спать|Я хочу) * P(хочу|Я) * P(Я)

Тогда (предсказываем предложение по условной вероятности последнего слова и остальных предложений):

$$
P\left(w_{1} w_{2} \ldots w_{n}\right)=\prod_{i} P\left(w_{i} \mid w_{1} w_{2} \ldots w_{i-1}\right)
$$

Однако число вероятностей вида $P\left(w_{i} \mid w_{1} w_{2} \ldots w_{i-1}\right)$ растет очень быстро. Поэтому используют некоторое предположение которое называется **марковковское приближение**. Формулируется оно так:

$$
P\left(w_{1} w_{2} \ldots w_{n}\right) \approx \prod_{i} P\left(w_{i} \mid w_{i-k} \ldots w_{i-1}\right)
$$

То есть мы считаем, что текущее слово зависит только от $k$ = (N-1) предыдущих.

$$
P\left(w_{i} \mid w_{1} w_{2} \ldots w_{i-1}\right) \approx P\left(w_{i} \mid w_{i-k} \ldots w_{i-1}\right)
$$


In [47]:
import numpy as np
from collections import Counter, defaultdict

from tqdm.notebook import tqdm
 

class NGramModel(object):
    """
    Структура этой реализации n-граммной модели следующая:
    self.ngrams – словарь, который на каждый (token_0, ..., token_(n-1)) – n-1 tuple из токенов
        хранит частоту появления следующего токена. Для подсчета числа токенов воспользуемся
        Counter
    self.tokenize_func – функция токенизации текста. С её помощью будем получать токены.
    """
    def __init__(self, n=2):
        self.ngrams = defaultdict(Counter)
        self.n = n
        self.tokenize_func = None
        
    def compute_ngrams(self, dataset):
        self.ngrams = defaultdict(Counter)
        for row in tqdm(dataset):
            ngram = [PAD] * self.n
            for token in row["features"]:
                ngram[:-1] = ngram[1:] # сдвигаем влево (все элементы до последнего заменяются от 2 до последнего)
                ngram[-1] = token
                self.ngrams[tuple(ngram[:-1])].update([ngram[-1]]) # по k предыдущих слов обновляем статистику последенего слова
            
    def get_log_probs(self, prefix, min_log_pr=-15):
        """
        Функция, которая будет возвращать логарифмы частот появления токенов
        """
        if len(prefix) < self.n - 1:
            prefix = [PAD] * (self.n - len(prefix) - 1) + prefix
        else:
            prefix = prefix[-self.n + 1:]
        possible_ends = self.ngrams[tuple(prefix)]
        sum_freq = np.log(sum(possible_ends[e] for e in possible_ends))
        return {e: np.log(possible_ends[e]) - sum_freq for e in possible_ends}
    
    def sample(self, prefix):
        possible_ends = self.get_log_probs(prefix)
        if len(possible_ends) > 0:
            end = np.random.choice(list(possible_ends.keys()), p=np.exp(list(possible_ends.values())))
            return end
        return EOS

In [49]:
n_gram_model = NGramModel(n=5)

In [50]:
n_gram_model.compute_ngrams(dataset["train"])

  0%|          | 0/30000 [00:00<?, ?it/s]

In [51]:
idx2token = {idx: token for token, idx in token2idx.items()}

In [52]:
prefix = ["def", "train", "("]
encoded_prefix = [token2idx[token] for token in prefix] # лист индексов
length=100

for i in range(length):
    cur_token = n_gram_model.sample(encoded_prefix)
    if cur_token == EOS:
        break
    encoded_prefix += [cur_token]


decoded_text = [idx2token[idx] for idx in encoded_prefix] # лист слов из листа индексов
print(" ".join(decoded_text))

def train ( cls , data , iterations = 100 , step = 1.0 , [UNK] = 1.0 ) : [UNK] global ret global resource ret = [ ] for filename in filenames : [UNK] ( filename , encoding , options ) : ret = dict ( ) conflicting = dict ( ) sites = list_sites ( ) if name in [UNK] : download ( name ) return ret else : [UNK] try : _data . decode ( 'UTF-8' ) try : if opts [ [UNK] ] ) ) return line_str . strip ( ) ) [UNK] = Timestamp ( datetime ( start_date


In [53]:
test_dataset = load_dataset(
    "json",
    data_files=[
        "test.jsonl",
    ],
)

Using custom data configuration default-d33f73aedb04cf32


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to C:\Users\BIT\.cache\huggingface\datasets\json\default-d33f73aedb04cf32\0.0.0\d75ead8d5cfcbe67495df0f89bd262f0023257fbbbd94a730313295f3d756d50. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
max_seq_len=128

test_dataset = test_dataset.map(
    lambda item: {
        "features": [encode(token) for token in item["code_tokens"]][:max_seq_len-1] + [EOS]
    }
)

  0%|          | 0/22176 [00:00<?, ?ex/s]

### Метрика качества (perplexity) (2 показывает сколько бит нужно, чтобы закодировать информацию) (если модель будет случайная, то perplexity будет равна размеру вокабуляра)

$$
P P(p):=2^{H(p)}=2^{-\sum_{x} p(x) \log _{2} p(x)}
$$

Можно использовать изменённую perxplexity:

$$
P P' (p):=e^{H(p)}=e^{-\sum_{x} p(x) \ln p(x)}
$$

$$
P P' (p):=e^{H(p)}=e^{-\frac{1}{n}\sum_{x} p(x) \ln p(x)}
$$

In [55]:
def count_perplexity(model, dataset, max_iter_num: int = 1000):
    entropy = 0
    iter_num = 0
    num_words = 0
    for item in tqdm(dataset, total=min(max_iter_num, len(dataset))):
        output_so_far = [item["features"][0]]

        for token in item["features"][1:]:
            num_words += 1
            try:
                log_probs = model.get_log_probs(output_so_far)
                entropy += -log_probs[token]
            except KeyError:
                entropy += np.log(-10)
            output_so_far.append(token)
        iter_num += 1
        if iter_num > max_iter_num:
            break
    mean_entropy = entropy / num_words
    return np.e ** mean_entropy

In [56]:
count_perplexity(n_gram_model, test_dataset["train"]) # в тесте есть примеры, которых нет в трейне (некоторая последовательность слов никогда не встречалась в трейне)

  0%|          | 0/1000 [00:00<?, ?it/s]

C:\Users\BIT\AppData\Local\Temp/ipykernel_4204/864722292.py:14: RuntimeWarning: invalid value encountered in log
  entropy += np.log(-10)
C:\Users\BIT\AppData\Local\Temp/ipykernel_4204/2198818106.py:38: RuntimeWarning: divide by zero encountered in log
  sum_freq = np.log(sum(possible_ends[e] for e in possible_ends))


nan

## CNN

Подаём в модель префикс с использованием спец.токнов, используем свёртки и получаем следущий токен

![](https://lena-voita.github.io/resources/lectures/lang_models/neural/cnn/cnn_main-min.png)



In [67]:
dataset.set_format(type="torch", columns=["features"]) # перевод из формата list в torch
test_dataset.set_format(type="torch", columns=["features"])

In [68]:
dataset['train']['features'][0]

tensor([   15,   427,     3,  4362,     6,     1,     7,    14,     6, 16755,
            7,    14,     6,     1,     7, 26799,     6,   315,     7,    27,
            4,     8,   272,     7,     9,    10,   134,     7,     9,    10,
            1,    22, 32600,    16,    45,     5,   707,     3,  4362,     4,
            8,    12,    18,    45,     5,    33,     5,   461,     3,    45,
            5,    33,     5,    64,     3,  4362,     6, 32600,     4,     4,
            8,   112,     1,    22,  4952,    16,     1,     3,    45,     5,
           33,     5,    64,     3,  4362,     6, 32600,     4,     4,     8,
          224,     7, 22186,     5,     1,     3,  4952,     4, 32601,     7,
        22186,     5, 26800,     3,   224,     4,    12,    44,     3, 32601,
            4,    67,    21,     8,     1,    12,   315,     8,   126,     3,
            1,     5,    30,     3,  4952,     6,     1,    12,    44,     3,
        32601,     4,   129,    21,    24,     1,     4,     4, 

In [106]:
def collate_fn(batch): # функция, которая применяется для каждого батча (в данном случае доппадинг каждого батча, чтобы получилсь матрица len_batch x max_len)
    batch = batch[0]
    max_len = max(len(f_t) for f_t in batch["features"])
    input_embeds = torch.zeros((len(batch["features"]), max_len), dtype=torch.long)
    for idx, row in enumerate(batch["features"]):
        input_embeds[idx][:len(row)] += row
    return {
        "features": input_embeds,
    }

In [107]:
from torch.utils.data import Sampler


class TextSampler(Sampler): # определяет какие примеры будем сэмплировать из датасета (борется с большими примерами которые могут не влезть в память)
    def __init__(self, sampler, batch_size_tokens=1e3): # максимальное кол-во токенов в батче 10^4
        self.sampler = sampler
        self.batch_size_tokens = batch_size_tokens

    def __iter__(self):
        batch = []
        max_len = 0
        for ix in self.sampler:
            row = self.sampler.data_source[ix]
            max_len = max(max_len, len(row["features"]))
            if (len(batch) + 1) * max_len > self.batch_size_tokens:
                yield batch # возвращает батч из функции, если его размер стал превышать допустимый (но продолжает выполнение функции снова)
                batch = [] # обнулили батч
                max_len = len(row["features"]) # переопределили максимальную длину строки
            batch.append(ix) # накапливаем батч
        if len(batch) > 0: # последний батч может быть длины меньше, однако нам тоже нужно его возвратить
            yield batch

    def __len__(self):
        return len(self.sampler)

In [108]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler, random_split


train_sampler = RandomSampler(dataset["train"])
valid_sampler = SequentialSampler(test_dataset["train"])

loaders = {
    "train": DataLoader(
        dataset["train"], 
        collate_fn=collate_fn, 
        sampler=TextSampler(train_sampler,)
    ),
    "valid": DataLoader(
        test_dataset["train"],
        collate_fn=collate_fn, 
        sampler=TextSampler(
            valid_sampler, 
        )
    )
}

In [109]:
import torch
import torch.nn as nn


class CNNLM(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, num_layers=3, kernel_size: int = 5):
        super().__init__()
        
        self.emb = nn.Embedding(vocab_size, emb_size)
        layers = []
        for layer_idx in range(num_layers):
            layers.append(nn.ZeroPad2d((kernel_size-1, 0, 0, 0))) # добавляем паддинги в начало (ks -1 из-за <bos>)
            if layer_idx == 0: # сначала emb_size -> hidden_size
                layers.append(nn.Conv1d(emb_size, hidden_size, kernel_size=kernel_size))
            else: # потом hidden_size -> hidden_size
                layers.append(nn.Conv1d(hidden_size, hidden_size, kernel_size=kernel_size))
        self.conv_layers = nn.Sequential(*layers)
        self.receptive_field = kernel_size + (kernel_size-1)*(num_layers-1) # для ks=3 и nl=3 rf=3+2+2=7
        self.pred = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_ids):
        embed = self.emb(input_ids) # получаем эмбеддинги
        embed = embed.permute(0, 2, 1) # транспонируем
        features = self.conv_layers(embed) # пропускаем через свёрточные слои
        features = features.permute(0, 2, 1) # транспонируем обратно
        logits = self.pred(features) # делаем предсказание
        return logits

In [110]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = CNNLM(len(tokens) + 3, 300, 100, num_layers=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss(ignore_index=PAD)

In [111]:
from tqdm.notebook import tqdm, trange


def train(
    num_epochs: int, 
    model: nn.Module,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    max_grad_norm: float = None
):
    for epoch in trange(num_epochs):
        pbar = tqdm(train_loader, leave=False, total=len(train_loader)//20)
        pbar.set_description("Train epoch")
        model.train()
        for batch in pbar:
            optimizer.zero_grad()
            features = batch["features"].to(device)
            predictions = model(features[:, :-1]) # не берём последний токен
            loss = criterion(
                predictions.reshape(-1, predictions.size(-1)),
                features[:, 1:].reshape(-1)
            )
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # обрезаем градиенты
            optimizer.step()
        model.eval()
        mean_loss = 0
        pbar = tqdm(valid_loader, leave=False, total=len(valid_loader)//100)
        pbar.set_description("Valid epoch")
        num_iter=0
        for batch in pbar:
            features = batch["features"].to(device)
            with torch.no_grad():
                predictions = model(features[:, :-1])
                loss = criterion(
                    predictions.reshape(-1, predictions.size(-1)),
                    features[:, 1:].reshape(-1)
                )
            mean_loss += loss.item()
            num_iter += 1
        mean_loss /= num_iter
        print(f"Epoch: {epoch}; mean loss: {mean_loss}; perplexity: {np.exp(mean_loss)}")
            

In [84]:
train(
    num_epochs=1,
    model=model, 
    train_loader=loaders["train"],
    valid_loader=loaders["valid"],
    criterion=criterion,
    optimizer=optimizer,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/221 [00:00<?, ?it/s]

Epoch: 0; mean loss: 3.737670251454728; perplexity: 42.00002659320553


# Perplexity (недоумение), чем меньше, тем лучше

In [85]:
torch.save(model,'CNNLM.pt')

In [83]:
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.synchronize()

C:\Users\BIT\.conda\envs\deep\lib\site-packages\torch\cuda\memory.py:231: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


![](https://lena-voita.github.io/resources/lectures/lang_models/neural/cnn/receptive_field-min.png)

На первом слое у rf ks токенов, на втором слое к rf добавляется ks-1 токенов, и т.д на каждом следущем слое к rf будет добавляться ks-1 токенов

Как увеличить receptive field? 

Добавить больше слоев.

Как обучать?

Добавить residual connections. (то есть хорошо будут обучаться не только последние слои, но и предыдущие)


![](https://lena-voita.github.io/resources/lectures/lang_models/neural/cnn/cnn_with_residual-min.png)



In [86]:
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.synchronize()

In [112]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from ipywidgets import interactive
from IPython import display

sns.set(style="whitegrid", font_scale=1.4)

sample = np.random.randn(10) # 10 нормальных чисел до softmax
def plot_temperature(T: float = 1.0):
    plt.figure(figsize=(12, 8))
    plt.title(f"Temperature = {T}")
    probs = np.exp(sample / T) / sum(np.exp(sample / T)) # softmax(sample/T) (10 вер-тей)
    plt.bar(range(10), probs)
    plt.xlabel("tokens")
    plt.ylabel("probs")
    plt.show()


v = interactive(
    plot_temperature, T=(0.02, 10)
)

при нулевой температуре останется один кандитат и модель полностью определяет следующее слово из-за чего модель может начать заикаться (повторять одно и тоже)

In [88]:
display.display(v) # при увеличении температуры распределение становится более равномерным (энтропия повышается) (поведение модели становится более случайным)

interactive(children=(FloatSlider(value=1.0, description='T', max=10.0, min=0.02), Output()), _dom_classes=('w…

In [113]:
from typing import List
from torch.distributions import Categorical

@torch.no_grad()
def generate(
    prefix, model, length: int = 100, receptive_field: int = 5, T: float = 1.
) -> List[int]:
    prefix = torch.from_numpy(prefix)
    prefix = prefix.unsqueeze(0).to(device)
    model.eval()
    for iter_idx in range(length):
        preds = model(prefix[:, -receptive_field:])
        probs = torch.softmax(preds[:, -1]/T, dim=-1)
        distribution = Categorical(probs)
        sampled = distribution.sample()
        if sampled.item() == EOS:
            break
        prefix = torch.cat((prefix, sampled.unsqueeze(0)), dim=1)
    return prefix

In [114]:
prefix = ["def", "train", "("]
encoded_prefix = np.array([token2idx[t] for t in prefix])


for t in np.logspace(0.002, 1, 10):
    generated = generate(
        encoded_prefix, 
        model, 
        receptive_field=model.receptive_field, 
        length=20,
        T=t-1
    )
    print(f"Temperature: {t-1}")
    print(" ".join([idx2token[idx] for idx in generated.cpu().numpy().flatten()]))

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.IntTensor instead (while checking arguments for embedding)

## LSTM

Позволяет сразу учитывать полное предложение (без использования большого кол-во свёрточных слоёв)

![](https://lena-voita.github.io/resources/lectures/lang_models/neural/rnn/rnn_simple-min.png)

In [100]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size, batch_first=True)
        self.pred = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, input_ids):
        embs = self.emb(input_ids) # получаем эмбеддинги
        output, _ = self.lstm(embs) # получаем выход модели (output, а не hidden)
        return self.pred(output) # получаем предсказания

In [101]:
model = LSTM(len(token2idx), 300, 50).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [102]:
train(
    num_epochs=1,
    model=model,
    train_loader=loaders["train"],
    valid_loader=loaders["valid"],
    criterion=criterion,
    optimizer=optimizer,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

C:\Users\BIT\.conda\envs\deep\lib\site-packages\datasets\formatting\formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(array, copy=False, **self.np_array_kwargs)


  0%|          | 0/221 [00:00<?, ?it/s]

Epoch: 0; mean loss: 3.3000318043386243; perplexity: 27.1135012339197


In [103]:
torch.save(model,'LSTM.pt')

In [ ]:
with open('/LSTM.pt') as f:
    model = torch.load(f)

## Методы генерации текста

### Greedy Search (жадный поиск) (на каждом шаге берём только тот токен вер-ть которого больше всего)

$$
w_t = \operatorname{argmax}_{w} P\left(w \mid w_{1: t-1}\right)
$$

![](https://huggingface.co/blog/assets/02_how-to-generate/greedy_search.png)

**Проблема**: Модель быстро начинает повторять одну и ту же фразу. (низкая темпрература)

### Beam search (лучевой поиск) (пускать лучи самые вероятные токены (в данном случае их 2))

Вероятность пути это произведение вер-тей, которые у нас по нему встречались

![](https://huggingface.co/blog/assets/02_how-to-generate/beam_search.png)

**Проблема**: Модель все еще выдает слишком предсказуемый текст, в отличии от человеческой речи.
![](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)

### Sampling (у каждого токена получаем распределение и его сэплируем)

$$
w_{t} \sim P\left(w \mid w_{1: t-1}\right)
$$

![](https://huggingface.co/blog/assets/02_how-to-generate/sampling_search_with_temp.png)

**Проблема**: страдает целостность текста. Некоторые фразы получаются слишком случайные.

### Top-K Sampling (берём топ k (10 .. 50) вариантов и уже это считать за распределение) (также можно набирать вероятность (то есть, например чтобы вер-ть была больше 0.9))

Для обоих картинок k = 6

![](https://huggingface.co/blog/assets/02_how-to-generate/top_k_sampling.png)

Еще можно использовать top-p sampling. Жадно набирать слова, пока их общая вероятность не станет p.


In [104]:
prefix = ["def", "train", "("]
encoded_prefix = np.array([token2idx[t] for t in prefix])

generated = generate(encoded_prefix, model) # пересмотреть как модель может получать предложение произвольной длины

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.IntTensor instead (while checking arguments for embedding)

In [105]:
prefix = ["def", "train", "("]
encoded_prefix = np.array([token2idx[t] for t in prefix])


for t in np.logspace(0.002, 1, 10):
    generated = generate(
        encoded_prefix, 
        model, 
        receptive_field=20, 
        length=20,
        T=t-1
    )
    print(f"Temperature: {t-1}")
    print(" ".join([idx2token[idx] for idx in generated.cpu().numpy().flatten()]))

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.IntTensor instead (while checking arguments for embedding)

## References



1.   [Заметки из курса ШАДа.](https://lena-voita.github.io/nlp_course/language_modeling.html)
2.   [Блогпост по теме генерации текста от huggingface.](https://huggingface.co/blog/how-to-generate) Пока не заморачивайтесь, что там за модель в примере. Мы ее подробно рамерем в одном из следующих занятий.

